In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window
import numpy as np
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/01 16:01:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/01 16:01:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
merchant_study_score = spark.read.parquet('../data/curated/merchant_info.parquet')
merchant_pd = merchant_study_score.toPandas()

In [4]:
merchant_pd

,Store_type,Revenue_levels,Take_rate,count_of_bigorder,Avg_amount_monthly,Avg_count_monthly,Order_avg_value,postcode,avg_prob_fraud_cus,prob_num_of_fraud,count_cus_per_mon,merchant_abn,fix_cus_num,avg_income_percentage,avg_age_percentage
0,"furniture, home furnishings and equipment shop...",e,5157.0522,0,28650.29,159.94,179.13,5582,13.103379,0.002961,150.11,10023283211,0,0.146046,0.488653
1,"cable, satellite, and other pay television and...",b,15788.5392,0,3741.36,143.50,26.07,6438,13.235562,0.004918,135.56,10142254217,0,0.304018,0.737510
2,"jewelry, watch, clock, and silverware shops",b,13733.0160,3,3121.14,0.28,11236.09,6421,18.309875,0.000000,0.28,10165489824,0,0.277595,0.583768
3,"watch, clock, and jewelry repair shops",b,5181.9145,0,1575.05,16.50,95.46,5356,11.873047,0.003378,16.44,10187291046,0,0.188470,0.540919
4,"music shops - musical instruments, pianos, and...",a,42902.2080,0,6777.60,18.78,360.94,9015,13.512474,0.002967,18.72,10192359162,0,0.342030,0.805485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4021,"opticians, optical goods, and eyeglasses",b,75731.2650,0,16829.17,831.22,20.25,6467,13.061586,0.002782,619.06,99938978285,9,0.251363,0.534333
4022,"books, periodicals, and newspapers",b,4362.9661,0,1376.33,6.17,223.19,6424,12.343478,0.000000,6.17,99974311662,0,0.297329,0.620632
4023,shoe shops,a,922784.8824,0,140454.32,1122.22,125.16,4346,13.011395,0.003059,762.78,99976658299,37,0.220345,0.597243
4024,motor vehicle supplies and new parts,a,20187.8820,0,2960.10,9.61,307.99,9013,12.604562,0.000000,9.50,99987905597,0,0.342030,0.805485


In [15]:
from pyspark.sql.functions import col
import scipy.stats as st
import pandas as pd

def convert_to_score(table_name, col_name):
    sorted_col = table_name[['merchant_abn', col_name]].sort_values(by=col_name, ascending=False)
    num_of_merchant = len(sorted_col)
    rank_list = [i for i in range(1, num_of_merchant+1)]
    sorted_col['rank'] = rank_list
    sorted_col['percentage above the merchants'] = 1-sorted_col['rank']/(num_of_merchant+1)
    sorted_col['z score'] = st.norm.ppf(sorted_col['percentage above the merchants'])
    sorted_col[col_name+'_raw_score'] = sorted_col['z score']*5.75+30
    return sorted_col[['merchant_abn', col_name+'_raw_score']]

numeric_features_list = ['Take_rate', 'count_of_bigorder', 'Avg_amount_monthly', 'Avg_count_monthly', 'Order_avg_value', 
    'avg_prob_fraud_cus', 'prob_num_of_fraud', 'count_cus_per_mon', 'fix_cus_num', 'avg_income_percentage', 
    'avg_age_percentage']

merchant_pd[['prob_num_of_fraud']] = -abs(merchant_pd[['prob_num_of_fraud']])
merchant_pd[['avg_prob_fraud_cus']] = -abs(merchant_pd[['avg_prob_fraud_cus']])

raw_score_df = merchant_pd[['merchant_abn', 'Store_type']]
# Revenue Level
reve = merchant_pd[['merchant_abn', 'Revenue_levels']]
reve[['Revenue_levels_raw_score']] = reve[['Revenue_levels']].replace(to_replace = ['a', 'b', 'c', 'd', 'e'], value = [10, 20, 30, 40, 50])
raw_score_df = pd.merge(raw_score_df, reve[['merchant_abn', 'Revenue_levels_raw_score']], how='inner', on = 'merchant_abn')


for feature in numeric_features_list:
    feature_raw_score = convert_to_score(merchant_pd, feature)
    raw_score_df = pd.merge(raw_score_df, feature_raw_score, how='inner', on = 'merchant_abn')


raw_score_df['raw_score_sum']= raw_score_df.iloc[:, 2:-1].sum(axis=1)
raw_score_df[['merchant_abn', 'raw_score_sum']].sort_values(by='raw_score_sum', ascending=False).head(20)
raw_score_df #原始表格

/var/folders/63/097df8mn6r1d0btnhwhjpkr40000gn/T/ipykernel_99390/1841927080.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reve[['Revenue_levels_raw_score']] = reve[['Revenue_levels']].replace(to_replace = ['a', 'b', 'c', 'd', 'e'], value = [10, 20, 30, 40, 50])


,merchant_abn,Store_type,Revenue_levels_raw_score,Take_rate_raw_score,count_of_bigorder_raw_score,Avg_amount_monthly_raw_score,Avg_count_monthly_raw_score,Order_avg_value_raw_score,avg_prob_fraud_cus_raw_score,prob_num_of_fraud_raw_score,count_cus_per_mon_raw_score,fix_cus_num_raw_score,avg_income_percentage_raw_score,avg_age_percentage_raw_score,raw_score_sum
0,10023283211,"furniture, home furnishings and equipment shop...",50,25.636938,33.581244,34.444645,35.122680,28.647244,29.897990,29.316572,35.106732,36.991952,20.395766,18.880817,359.141762
1,10142254217,"cable, satellite, and other pay television and...",20,29.525228,24.161365,29.154076,34.795600,19.965216,27.786527,24.476250,34.770307,24.671779,32.485848,36.736513,301.792196
2,10165489824,"jewelry, watch, clock, and silverware shops",20,29.193855,41.335886,28.823255,18.204477,42.206002,18.562399,31.619226,18.430002,27.320151,31.530044,30.947367,307.225296
3,10187291046,"watch, clock, and jewelry repair shops",20,25.651244,27.439275,26.776952,29.456957,26.172085,38.421357,28.084316,29.460552,27.316161,24.764558,25.964075,303.543457
4,10192359162,"music shops - musical instruments, pianos, and...",10,31.999122,27.435322,30.661806,29.801319,31.585738,25.593857,29.305757,29.812063,27.312169,34.463972,37.423768,307.971126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4021,99938978285,"opticians, optical goods, and eyeglasses",20,33.456075,32.221183,32.942037,40.255401,17.793998,30.759184,29.665163,40.255401,40.002125,29.381411,25.409154,346.731977
4022,99974311662,"books, periodicals, and newspapers",20,24.898576,32.217328,25.931960,26.963733,29.485712,36.603250,35.080238,26.943141,31.976325,32.283015,33.098162,322.383277
4023,99976658299,shoe shops,10,42.105688,32.213473,41.261864,41.237599,27.244110,31.548592,29.049005,41.261864,41.189662,27.427413,31.866570,364.539269
4024,99987905597,motor vehicle supplies and new parts,10,30.016106,32.209619,28.720727,28.095662,30.853161,35.101424,35.074952,28.069176,31.972529,34.517381,37.326006,324.630737


In [11]:
reve = merchant_pd[['merchant_abn', 'Revenue_levels']]
reve[['Revenue_levels_raw_score']] = reve[['Revenue_levels']].replace(to_replace = ['a', 'b', 'c', 'd', 'e'], value = [10, 20, 30, 40, 50])
reve

/var/folders/63/097df8mn6r1d0btnhwhjpkr40000gn/T/ipykernel_99390/462840560.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reve[['Revenue_levels_raw_score']] = reve[['Revenue_levels']].replace(to_replace = ['a', 'b', 'c', 'd', 'e'], value = [10, 20, 30, 40, 50])


,merchant_abn,Revenue_levels,Revenue_levels_raw_score
0,10023283211,e,50
1,10142254217,b,20
2,10165489824,b,20
3,10187291046,b,20
4,10192359162,a,10
...,...,...,...
4021,99938978285,b,20
4022,99974311662,b,20
4023,99976658299,a,10
4024,99987905597,a,10
